In [1]:
from PIL import Image, ImageDraw 
from IPython.display import Image as DImage
import sys, os, hashlib

In [2]:
#INTERFACE
pix_mode   = 1 #pixelhunt
hash_mode  = 1
black_mode = 0 #ingore - only black pixels count
bin_ratio  = 200
basewidth  = 8

#init directories
if not os.path.exists(os.getcwd()+'/examples/'):
    os.mkdir('examples')
if not os.path.exists(os.getcwd()+'/binary/'):
    os.mkdir('binary')
if not os.path.exists(os.getcwd()+'/resized/'):
    os.mkdir('resized')
examples_path = os.getcwd()+'/examples/'
binary_path = os.getcwd()+'/binary/'
resized_path = os.getcwd()+'/resized/'

#global variables
names = []
values = []
distances = []
RESULT = ''

In [3]:
input_address = input('Input image address:') #input must be in ""

Input image address:"test.jpg"


In [4]:
def address_helper(input_img_addr):
    input_img_addr = input_img_addr.replace('.jpg','.png')
    input_img_addr = input_img_addr.replace('.gif','.png')
    return input_img_addr

In [5]:
def binarization(input_img_addr):
    input_img = Image.open(input_img_addr) 
    draw = ImageDraw.Draw(input_img)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    if ".gif" not in input_img_addr:
        for i in range(input_width):
            for j in range(input_height):
                r = input_pixels[i, j][0]
                g = input_pixels[i, j][1]
                b = input_pixels[i, j][2]
                s = r + g + b
                if (s > bin_ratio):
                    r, g, b = 255, 255, 255
                else:
                    r, g, b = 0, 0, 0
                draw.point((i, j), (r, g, b))
    input_img_addr = address_helper(input_img_addr)
    #if "/examples/" not in input_img_addr:
    #    input_img_addr = os.getcwd()+'/binary/'+input_img_addr
    #else:
    input_img_addr = input_img_addr.replace('/examples/','/binary/')
    input_img_addr = input_img_addr.replace('/resized/','/binary/')
    input_img.save(input_img_addr, "PNG")
    del draw

In [6]:
def gethash(input_img_addr):
    input_img = Image.open(input_img_addr)
    return (hashlib.md5(input_img.tobytes()).hexdigest())

In [7]:
def hamdist(str1, str2, prevMin=None):
    diffs = 0
    if len(str1) != len(str2):
        return max(len(str1),len(str2))
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
            if prevMin is not None and diffs>prevMin:
                return None
    return diffs

In [8]:
def resize(input_img_addr, basewidth):
    img = Image.open(input_img_addr)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize))#,Image.ANTIALIAS) 
    #if "/examples/" not in input_img_addr:
    #    input_img_addr = os.getcwd()+'/resized/'+input_img_addr
    input_img_addr = input_img_addr.replace('/examples/','/resized/')
    input_img_addr = input_img_addr.replace('/binary/','/resized/')
    img.save(input_img_addr, "PNG")

In [9]:
def compare1to1(input_img_addr,example_addr):
    example_addr = address_helper(example_addr)
    input_img_addr = address_helper(input_img_addr)
    if pix_mode == 1:
        input_img_addr = os.getcwd()+'/binary/'+input_img_addr
        input_img = Image.open(input_img_addr)
        input_pixels = input_img.load()
        example = Image.open(example_addr)
            
        true = 0 
        false = 0
        black = 0
        pixels = 0
    
    
        for i in xrange(0, input_img.size[0]):
                for j in xrange(0, input_img.size[1]):
                    if input_pixels[i, j][0] == 255:
                        black+=1
                    input_pixel = input_img.getpixel((i, j))
                    example_pixel = example.getpixel((i, j))
                    if input_pixel != example_pixel:                        
                        false+=1
                    else:
                        if black_mode == 'ignore' and input_pixels[i, j][0] == 0:
                            break
                        true+=1    
        values.append(true)

        
    if hash_mode == 1:
        resize(input_img_addr, basewidth)
        resize(example_addr, basewidth)
        example_addr = example_addr.replace('/binary/','/resized/')
        input_img_addr = input_img_addr.replace('/binary/','/resized/')
        input_hash = gethash(input_img_addr)
        
        
        example_hash = gethash(example_addr)
        
        distances.append(hamdist(input_hash, example_hash))
        
    example_addr = example_addr.replace('.jpg','')
    example_addr = example_addr.replace('.gif','')
    example_addr = example_addr.replace('.png','')
    example_addr = example_addr.replace(os.getcwd()+'/binary/','')
    example_addr = example_addr.replace(os.getcwd()+'/resized/','')
    names.append(example_addr)   

In [10]:
def compare1toN(input_img_addr):
    global names
    global values
    global distances
    names = []
    values = []
    distances = []
    examples = os.listdir(examples_path)
    #if mode == 'pixelhunt':
    for i in range(len(examples)):
        binarization(examples_path+examples[i])
        compare1to1(input_img_addr,binary_path+examples[i])
    if hash_mode == 1:
        dmin=distances[0]
        for i in range(len(distances)):
            if distances[i] is not None and distances[i] < dmin:
                dmin = distances[i]

In [11]:
def print1toN():
    if pix_mode == 1:
        print('Pixel mode:')
        for i in range(len(names)):
            print(names[i],values[i])
    if hash_mode == 1:
        print('Hash mode:')
        for i in range(len(names)):
            print(names[i],distances[i])

In [12]:
def result1toN():
    if pix_mode == 1:
        global RESULT 
        RESULT = RESULT + names[values.index(max(values))]
        return names[values.index(max(values))]
    if hash_mode == 1:
        return names[distances.index(min(distances))]

In [13]:
binarization(input_address)
compare1toN(input_address)
print1toN()
print(result1toN())

Pixel mode:
('test', 47718)
('ctest', 38706)
('compare', 44884)
('test2', 0)
('a', 47718)
Hash mode:
('test', 0)
('ctest', 32)
('compare', 27)
('test2', 30)
('a', 0)
test
